In [1]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import numpy as np



Autosaving every 180 seconds


In [2]:
# LOAD DATA FROM .NPZ FILE
def load_data(fname, 
              return_all_sessions=False,
              return_good_sessions=False,
              return_best_sessions=True):
              
    data = np.load(fname,allow_pickle=True)
    data_04 = data['data_04']
    data_04_random = data['data_04_random'] 
    data_04_lockout = data['data_04_lockout']
    data_04_lockout_random= data['data_04_lockout_random']
    session_list = data['session_list']
    ctr_list = data['ctr_list']

    selected_sessions = data['good_sessions']
    best_sessions = data['best_sessions']
    
    # MAKE PRINTOUT TABLE
    print ("rec id,          rec name,           all rewarded trials,   10sec lockout rewarded trials (*** good sessions; ####### best 3 sessions")
    for ctr,recording in enumerate(session_list):
        prefix = '       '
        if ctr in selected_sessions:
            if ctr in best_sessions:
                prefix="#######"
            else:
                prefix='    ***'

        print (prefix,ctr, "     ", recording,"    ", data_04[ctr].shape, "        ", data_04_lockout[ctr].shape)

    print ("")
    print ("")
        
    # RETURN  SESSIONS
    if return_all_sessions==True:
        print ("Returning all sessions")
        idx = ctr_list
    elif return_good_sessions==True:
        print ("Returning good sessions")
        idx = selected_sessions
    else:
        print ("Returning best sessions")
        idx = best_sessions
        
    return ctr_list[idx], \
           session_list[idx], \
           data_04[idx], \
           data_04_random[idx], \
           data_04_lockout[idx], \
           data_04_lockout_random[idx]
    

In [3]:
#########################################################
###################### LOAD DATA ########################
#########################################################

# INPUT ANIMAL DATA FILE NAME
fname = '/media/cat/4TBSSD/yuki/time_courses_all_rewarded_330sessions_no_lockout/AQ2.npz'

# LOAD SELECTED SESSIONS ONLY
(ctr_list, selected_sessions, data_04, data_04_random,
                data_04_lockout,data_04_lockout_random) = load_data(fname, 
                                                                    return_all_sessions=False,
                                                                    return_good_sessions=False,
                                                                    return_best_sessions=True)

#
print ("selected_sessions: ", selected_sessions)
print ("")
#
print ("Dimensions of the first selected session without lockout: ", data_04[0].shape)
    

rec id,          rec name,           all rewarded trials,   10sec lockout rewarded trials (*** good sessions; ####### best 3 sessions
        0       AQ2am_Dec9_30Hz      (0,)          (0,)
        1       AQ2am_Dec10_30Hz      (0,)          (0,)
        2       AQ2pm_Dec10_30Hz      (0,)          (0,)
        3       AQ2am_Dec11_30Hz      (7, 35, 601)          (7, 35, 601)
        4       AQ2pm_Dec14_30Hz      (10, 35, 601)          (9, 35, 601)
        5       AQ2am_Dec14_30Hz      (0,)          (0,)
        6       AQ2pm_Dec16_30Hz      (27, 35, 601)          (22, 35, 601)
        7       AQ2am_Dec17_30Hz      (21, 35, 601)          (15, 35, 601)
        8       AQ2pm_Dec17_30Hz      (5, 35, 601)          (5, 35, 601)
        9       AQ2am_Dec18_30Hz      (7, 35, 601)          (7, 35, 601)
        10       AQ2pm_Dec18_30Hz      (5, 35, 601)          (5, 35, 601)
        11       AQ2am_Dec21_30Hz      (2, 35, 601)          (2, 35, 601)
        12       AQ2am_Dec22_30Hz      (0,)     

In [12]:
# select session
session_id = 2

data_selected = data_04[session_id]
print ("# trials, ROIs, # time points:", data_selected.shape)



# trials, ROIs, # time points: (100, 35, 601)


## 3 VISUALIZE PCA 

In [3]:
# LOAD SOME 4D STACK OVER TRIALS

data_selected = np.load('/media/cat/4TBSSD/yuki/IA2/tif_files/IA2pm_Feb3_30Hz/IA2pm_Feb3_30Hz_code_04_trial_ROItimeCourses_10sec_all_brain.npy')
print (data_selected.shape)

# SUBSELECT ONLY PART OF DATASET
if False:
    data_selected = data_selected[:,:data_selected.shape[1]//2]

print ("Final data shape: ", data_selected.shape)



(38, 601, 128, 128)
Final data shape:  (38, 601, 128, 128)


In [4]:
#########################################
######## LOAD IMAGING MASK ##############
#########################################
# mask represents the frame areas with neural activity (everything else is outside the brain)
mask = np.int32(np.loadtxt('/media/cat/4TBSSD/yuki/IA2/genericmask.txt'))
mask_brain = np.ones((128,128),'float32')
for k in range(mask.shape[0]):
    mask_brain[mask[k][0],mask[k][1]]=np.nan
plt.imshow(mask_brain)
plt.show()

In [5]:
###############################################################
################ LOAD MASKWARP WITH ROIS ######################
###############################################################
# mask represents
mask_roi = np.load('/media/cat/4TBSSD/yuki/IA2/tif_files/IA2pm_Apr22_Week2_30Hz/IA2pm_Apr22_Week2_30Hz_aligned_maskwarp.npy')
mask_roi = np.load('/home/cat/maskwarp.npy')

plt.imshow(mask_roi)
plt.show()

In [6]:
###########################################
########### FUNCTION TO COMPUTE PCA #######
###########################################
def get_PCA(data_selected,
            times,
           plotting=False):
    
    from sklearn.decomposition import PCA

    if False:
        #stack_out = np.hstack(data_selected[:,:,200:400].transpose(1,0,2))
        #stack_out = np.vstack(data_selected[:,:,200:400])

        #stack_out = data.reshape(data.shape[0],-1)
        pass
    # compute PCA on average trace
    elif True:
        #stack_out = np.mean(data_selected,axis=0)
        stack_out = np.mean(data_selected,axis=0).reshape(data_selected.shape[1],-1)[times]

    elif False:
        # sum over all ROIs
        stack_out = np.sum(data_selected,axis=1)

    print ("stack_out: ", stack_out.shape)
    # run PCA on all data

    pca = PCA(n_components=20)
    pca_model = pca.fit(stack_out)

    PCS = pca_model.transform(stack_out)
    print ("PCS out: ", PCS.shape)
    print ("DONE PCA Block")
    
    # plot PCA dynamics
    if plotting:
        fig=plt.figure()
        plt.scatter(PCS[:,0], PCS[:,1],alpha=.1)
        plt.show()
        
    # plot explained variance
    if plotting:
        fig=plt.figure()
        expl_variance = pca_model.explained_variance_[:10]
        plt.scatter(np.arange(expl_variance.shape[0]), expl_variance/expl_variance.sum(0),s=1000)
        plt.ylim(0,1)
        plt.ylabel("Explained Variance",fontsize=40)
        plt.xlabel("Principal component",fontsize=40)
        plt.tick_params(axis = 'both', which = 'major', labelsize = 40)
        plt.show()

    # 
    print ("components: ", pca_model.components_.shape)
    print ("singular values: ", pca_model.singular_values_[:10]**2)

    return PCS, pca_model

def pca_2d(pca_model, mask_brain):
    loadings = pca_model.components_.T #* np.sqrt(pca_model.explained_variance_)
    print (loadings.shape)

    # 
    plt.title("First 20 Loadings",fontsize=40)
    for k in range(6):
        ax = plt.subplot(2,3,k+1)
        plt.title("Comp: "+str(k), fontsize=20)

        temp = loadings[:,k].reshape(128,128)
        #vmax = np.max(np.abs(temp))
        #plt.imshow(temp*mask_original)
        #print ("max: ", np.max(temp)*pca_model.singular_values_[k]**2, 
        #       "min: ", np.min(temp)*pca_model.singular_values_[k]**2)
        plt.imshow(temp*mask_brain)
        plt.xticks([])
        plt.yticks([])

    #plt.suptitle("PC component * explained variance ")
    plt.show()
    
 

def fix_maskwarp():

    # load REGISTERED MAP IN 679 x 586 resolution
    dorsalMaps_unique = np.load('/home/cat/dorsalMaps_name.npy')
    print ("dorsaml maps: ", dorsalMaps_unique.shape)
    print ("areas: ", np.unique(dorsalMaps_unique).shape)

    # load REGISTERED MAP IN 128 x 128 resolution
    # NOTE: This map contains average errors that need to be removed:
    # so replace the values which are not in the dorsalMap list above with most frequent nearby values;

    # fix mean maskwarp values
    maskwarp = np.load('/home/cat/maskwarp.npy')
    #maskwarp = mask

    #print ("scaled/blended areas (due to affine transofrmation) pre fixed: ", np.unique(maskwarp).shape)
    #maskwarp_unique_ids = np.unique(maskwarp)
    maskwarp_unique_ids = np.unique(maskwarp)

    # work in 1D vectors easier to mask
    maskwarp1D = mask.reshape(-1)

    areas = np.unique(maskwarp) 
    print (" # of areas: ", areas.shape)

    # work in 1D vectors easier to mask
    maskwarp1D = maskwarp.reshape(-1)

    trial_courses = []
    area_ids = []

    data1D = np.float32(data_selected[0].reshape(data_selected[0].shape[0],-1))
    for id_ in areas:
        idx = np.where(maskwarp1D==id_)[0]
        # only keep areas that have at least 10 pixels
        if idx.shape[0]<=10:
            #print ("skipping ", "Area: ", id_)

            idx = np.where(maskwarp==id_)
            #print ("idx: ", idx)

            idx2 = np.where(maskwarp.flatten==id_)[0]
            for k in range(len(idx[0])):
                maskwarp[idx[0][k],idx[1][k]] = np.nan

            for k in range(len(idx[0])):
                temp = maskwarp[max(0,idx[0][k]-1): idx[0][k]+2, max(0,idx[1][k]-1): idx[1][k]+2].ravel()
                uniques = np.unique(temp, return_counts=True)
                idx4 = np.argmax(uniques[1])
                val = uniques[0][idx4]  # Replace values with most frequent nearby vals;
                maskwarp[idx[0][k],idx[1][k]] = val

    # ax = plt.subplot(1,2,2)
    # plt.imshow(maskwarp)
    print ("Post fix # of areas ", np.unique(maskwarp).shape)
    print ("area ids: ", np.unique(maskwarp))
    # plt.show()
    
    return maskwarp


def get_ROI_names(maskwarp):
    names_selected = np.int32(np.unique(maskwarp))

    # load names/labels for data
    ids = np.load('/home/cat/dorsalMaps_id.npy').squeeze()
    print ("All Allen IDS: ", ids.shape[0])
    import csv
    names = []
    with open('/home/cat/dorsalMaps_name.txt', newline='') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
        for row in spamreader:
            names.append(row)

    names.pop(0)

    print ("All Allen area names: ", len(names))

    for name_selected in names_selected:
        print (name_selected, "  selected area: ", names[name_selected])

    # GROUP AREAS BY REGION
    area_group_ids = [[8],                        # frontal
              [15,16,20,21],              # motor
              [29,36,43,50,57,64,71,78],  # somatosensory
              [100, 114],                 # auditory 
              [129,136,143,150,157,164],  # visual areas
              [249,255,261],              # restrosplenial          
             ]

    area_group_names = ['frontal','motor','somatosensory','auditory','visual','retrosplenial']
    
    return area_group_ids, area_group_names


def bar_graphs(pca_model,
              maskwarp1D,
              area_group_ids,
              area_group_names,
              times,
              xlabel_flag):
    
    # load the components
    loadings = pca_model.components_.T #* np.sqrt(pca_model.explained_variance_)
    print (loadings.shape)

    area_bars = np.zeros(6, 'float32')
    # loop over first 6 loadings:
    for k in range(6):
        temp = loadings[:,k]
        #print ("temp: ", temp.shape)
        #print ("maskwarp1d: ", maskwarp1D.shape)
        for ctr,area in enumerate(area_group_ids):

            area_pixels = []
            for id_ in area:
                idx = np.where(maskwarp1D==id_)[0]
                area_pixels.extend(idx)

            #print (names[area[0]], area, len(area_pixels))
            area_pixels = np.int32(area_pixels)

            #print (ctr, area_pixels, k)
            #area_bars[ctr] = area_bars[ctr]+np.abs(temp[area_pixels]).sum(0)*pca_model.singular_values_[k]**2
            area_bars[ctr] = area_bars[ctr]+temp[area_pixels].sum(0)*pca_model.singular_values_[k]**2

    plt.bar(np.arange(6), area_bars, 0.9)
    plt.xticks(np.arange(6), area_group_names, rotation=90)
    
    start_time = round(-10+times[0]/30.,1)
    end_time = round(-10 + times[-1]/30.,1)
    
    plt.title(str(start_time)+ " ... " +str(end_time) + " sec",fontsize=20 )
    
    if xlabel_flag==False:
        plt.xticks([])
        
    plt.show()
    

In [7]:
##################################################
###### FIX MASKWARP TO 35 ROIS #############
##################################################
# this function reduces the 262+ ROIs to the 35 that we selected tofr this project
maskwarp = fix_maskwarp()
maskwarp1D = maskwarp.ravel()

# 
area_group_ids, area_group_names = get_ROI_names(maskwarp)

dorsaml maps:  (679, 586)
areas:  (40,)
 # of areas:  (262,)
Post fix # of areas  (35,)
area ids:  [  0.   8.  15.  16.  20.  21.  29.  36.  43.  50.  57.  64.  71.  78.
 100. 114. 129. 136. 143. 150. 157. 164. 171. 178. 186. 198. 249. 255.
 261. 268. 275. 282. 300. 301. 653.]
All Allen IDS:  839
All Allen area names:  839
0   selected area:  ['"root"']
8   selected area:  ['"Frontal', 'pole,', 'layer', '2/3"']
15   selected area:  ['"Primary', 'motor', 'area,', 'Layer', '2/3"']
16   selected area:  ['"Primary', 'motor', 'area,', 'Layer', '5"']
20   selected area:  ['"Secondary', 'motor', 'area,', 'layer', '1"']
21   selected area:  ['"Secondary', 'motor', 'area,', 'layer', '2/3"']
29   selected area:  ['"Primary', 'somatosensory', 'area,', 'nose,', 'layer', '2/3"']
36   selected area:  ['"Primary', 'somatosensory', 'area,', 'barrel', 'field,', 'layer', '2/3"']
43   selected area:  ['"Primary', 'somatosensory', 'area,', 'lower', 'limb,', 'layer', '2/3"']
50   selected area:  ['"Primary

## START PROCESSING DATA

In [8]:
###############################################################
####################### COMPUTE PCA ###########################
###############################################################
# select which times to focus on;
times = np.arange(100)
times = np.arange(0,601,1)

xlabel_flag=False
    
ctr=0
chunk = 600
for k in range(0,600,chunk):
    times = np.arange(k,k+chunk,1)
    #print (times)
    PCS, pca_model = get_PCA(data_selected,times)
    # COMPUTE BAR GRAPHS FOR CONTRIBUTION FOR EACH AREA

    #if ctr4:
    xlabel_flag=True
        
    #ax=plt.subplot(4,5,ctr+1)
    bar_graphs(pca_model,
              maskwarp1D,
              area_group_ids,
              area_group_names,
              times,
              xlabel_flag)
    ctr+=1

plt.suptitle("PCA loadings for 6 ROIs based on chunk of time",fontsize=20)
plt.show()

stack_out:  (600, 16384)
PCS out:  (600, 20)
DONE PCA Block
components:  (20, 16384)
singular values:  [238.16017166 109.33485089  58.43092383  16.82101116   7.21160379
   3.45167293   3.07616673   2.4254532    1.3345102    1.06870049]
(16384, 20)


In [46]:
##################################################
############### COMPUTE PCA ON SELECTED TIMES ####
##################################################
times = np.arange(0,601,1)
#times = np.arange(210,390,1)

PCS, pca_model = get_PCA(data_selected,times)


stack_out:  (601, 16384)
PCS out:  (601, 20)
DONE PCA Block
components:  (20, 16384)
singular values:  [238.2055793  109.4893688   58.43522325  16.82463307   7.21540273
   3.46351233   3.07646678   2.42631526   1.35095475   1.06898851]


In [10]:
fig=plt.figure()
expl_variance = pca_model.explained_variance_[:10]
plt.scatter(np.arange(expl_variance.shape[0]), expl_variance/expl_variance.sum(0),s=1000)
plt.ylim(0,1)
plt.ylabel("Explained Variance",fontsize=40)
plt.xlabel("Principal component",fontsize=40)
plt.tick_params(axis = 'both', which = 'major', labelsize = 40)
print (expl_variance/expl_variance.sum(0))
plt.show()

[0.53966019 0.24774783 0.13240183 0.03811565 0.01634117 0.00782133
 0.00697045 0.00549597 0.00302394 0.00242163]


In [29]:
##################################################
############ PLOT PCs vs. TIME ###################
##################################################
# 
print (PCS.shape)
ax=plt.subplot(1,1,1)
time = np.arange(PCS.shape[0])/30.-10.
for k in range(6):
    plt.plot(time,
             PCS[:,k],#*pca_model.singular_values_[k]**2, 
             linewidth=5, label="PC "+str(k))
plt.xlabel("Time (sec)",fontsize=20)
ax.tick_params(axis="both", labelsize=20)
plt.legend(fontsize=20)
plt.show()

(601, 20)


In [24]:
##################################################
############ SHOW PCA 2D COMPONENTS ##############
##################################################
if True:
    pca_2d(pca_model, mask_brain)

(16384, 20)
max:  4.4019629888327225 min:  -0.8054539727785591
max:  0.47793700991408783 min:  -0.9970392136506078
max:  0.6083760401006643 min:  -0.6758098874580086
max:  0.1610131270573169 min:  -0.157359091168402
max:  0.1388686462927681 min:  -0.10443293794827391
max:  0.02872396814146379 min:  -0.03126072273532866


In [47]:
##################################################
############ PLOT PC1 vs. PC2 ####################
##################################################
ax=plt.subplot(111)
plt.scatter(PCS[:,0]*pca_model.singular_values_[0]**2, 
         PCS[:,1]*pca_model.singular_values_[1]**2,
        s=200, c=np.arange(PCS.shape[0]))

# plot t=0
plt.scatter(PCS[PCS.shape[0]//2,0]*pca_model.singular_values_[0]**2, 
         PCS[PCS.shape[0]//2,1]*pca_model.singular_values_[1]**2,
         s=400, c='red')

plt.scatter(PCS[0,0]*pca_model.singular_values_[0]**2, 
         PCS[0,1]*pca_model.singular_values_[1]**2,
         s=400, c='red')

plt.scatter(PCS[-1,0]*pca_model.singular_values_[0]**2, 
         PCS[-1,1]*pca_model.singular_values_[1]**2,
         s=400, c='red')

#
plt.text(PCS[0,0]*pca_model.singular_values_[0]**2, 
         PCS[0,1]*pca_model.singular_values_[1]**2+2,
         't='+str(round(times[0]/30.-10,1))+'sec', fontsize=20)

#
plt.text(PCS[PCS.shape[0]//2,0]*pca_model.singular_values_[0]**2, 
         PCS[PCS.shape[0]//2,1]*pca_model.singular_values_[1]**2+2,
         't=0sec', fontsize=20)

#
plt.text(PCS[-1,0]*pca_model.singular_values_[0]**2, 
         PCS[-1,1]*pca_model.singular_values_[1]**2+2,
         't='+str(round(times[-1]/30.-10,1))+'sec', fontsize=20)

plt.title(str(round(times[0]/30.-10,1))+"sec .."+str(round(times[-1]/30-10.,1))+"sec")
plt.xlabel("PC0", fontsize=20)
plt.ylabel("PC1", fontsize=20)
ax.tick_params(axis="both", labelsize=20)

plt.show()



In [119]:
# COMPUTE BAR GRAPHS FOR CONTRIBUTION FOR EACH AREA

bar_graphs(pca_model,
          maskwarp1D,
          area_group_ids,
          area_group_names,
          times)

(16384, 20)


In [ ]:
# load names/labels for data
ids = np.load('/home/cat/dorsalMaps_id.npy').squeeze()
print ("All Allen IDS: ", ids.shape[0])
import csv
names = []
with open('/home/cat/dorsalMaps_name.txt', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in spamreader:
        names.append(row)
        
print ("All Allen area names: ", len(names[1:]))



In [ ]:


def sum_pixels_in_registered_mask(data, maskwarp):

    #print (" # of trials: ", data.shape[0])

    areas = np.unique(maskwarp) 
    #print (" # of areas: ", areas.shape)
    
    # work in 1D vectors easier to mask
    maskwarp1D = maskwarp.reshape(-1)

    trial_courses = []
    area_ids = []
    for k in range(data.shape[0]):
        #if k%10==0:
        #    print ("computing trial: ", k)
        time_courses_local = []

        # convert to 1D vector to mask faster
        #data1D = np.float32(data[k].reshape(181,-1))
        data1D = np.float32(data[k].reshape(data[k].shape[0],-1))
        for id_ in areas:
            idx = np.where(maskwarp1D==id_)[0]

            # only keep areas that have at least 10 pixels
            if idx.shape[0]>10:
                #print ("Area: ", id_)
                area_ids.append(id_)#print ("Areas: ", id_)
                #print (data1D[:,idx].shape)

                temp = data1D[:,idx]

                if False:
                    # compute DFF
                    F0 = np.nanmean(temp,axis=0)
                    dFF = (data1D[:,idx]-F0)/F0
                else:
                    # skip dFF computation;
                    dFF = temp
                
                # save average of all pixesl post DFF
                time_courses_local.append(np.nanmean(dFF, axis=1))

            #all_times.append(time_courses_local)
        trial_courses.append(time_courses_local)

    area_ids = np.int32(np.unique(area_ids))
    trial_courses = np.float32(trial_courses)
    #print ("# trials, # areas, # times: ", trial_courses.shape)
    #print ("area ids: ", area_ids.shape)

    return area_ids, trial_courses

In [15]:
data = np.load('/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/training_images/classification_output.npz',
              allow_pickle=True)
y = data['y_test']
vals = data['vals']
frames_array = data['frames_array']
classes = data['classes']
print (y)

[[    0.]
 [    0.]
 [    0.]
 ...
 [12466.]
 [12466.]
 [12466.]]


In [19]:
print ("tracks: ", vals[:10], vals.shape)
print ("predicted ids: ", classes[:10], classes.shape)
print ("frame #: ", frames_array[:10], np.hstack(frames_array).shape)

tracks:  [5395. 5395. 5395. 5395. 5395. 2645. 2645. 5748. 5748. 5748.] (292353,)
predicted ids:  [4 4 4 4 4 2 2 1 1 0] (292353,)
frame #:  [list([40007, 40006, 40010, 40008, 40009]) list([26195, 26194])
 list([43498, 43497, 43507, 43501, 43506, 43505, 43499, 43500, 43502, 43504, 43503])
 list([37913, 37912])
 list([68880, 68950, 68894, 68839, 68859, 68832, 68891, 68947, 68937, 68869, 68903, 68857, 68879, 68848, 68919, 68913, 68956, 68962, 68884, 68931, 68907, 68850, 68968, 68868, 68904, 68846, 68888, 68940, 68856, 68808, 68892, 68957, 68918, 68943, 68820, 68806, 68890, 68829, 68866, 68945, 68863, 68917, 68871, 68818, 68942, 68911, 68867, 68875, 68824, 68949, 68897, 68900, 68912, 68862, 68952, 68813, 68901, 68842, 68865, 68958, 68816, 68887, 68939, 68861, 68826, 68946, 68812, 68895, 68864, 68807, 68838, 68923, 68963, 68920, 68858, 68944, 68819, 68855, 68837, 68898, 68872, 68873, 68851, 68833, 68927, 68936, 68821, 68843, 68876, 68966, 68847, 68934, 68825, 68893, 68844, 68845, 68948, 6882